neoLUTO Data Preparation
========================

Required input files:

* cell_LU_mapping.h5
* cell_zones_df.h5
* cell_livestock_data.h5
* SA2_crop_data.h5
* cell_biophysical_df.h5
* NLUM_SPREAD_LU_ID_Mapped_Concordance.h5
* SA2_climate_damage_mult.h5

* tmatrix-categories.csv
* tmatrix-cat2lus.csv

In [9]:
import numpy as np
import pandas as pd

From `cell_LU_mapping.h5` the following are obtained:
* CELLS_SA2 -- CELL_ID-SA2_ID concordance table.
* LANDUSES -- lexicographically ordered list of land-uses (strings).
* LUMAP -- present (2010) land-use mapping.
* LMMAP -- present (2010) land-management mapping.

In [10]:
lmap = pd.read_hdf('cell_LU_mapping.h5')

FileNotFoundError: File cell_LU_mapping.h5 does not exist

## CELLS_SA2 (concordance table) ##

In [3]:
# Make sure the CELL_ID starts at zero, rather than one.
lmap.reset_index(inplace=True)
lmap = lmap.rename(columns={'CELL_ID': 'CELL_ID_PLUSONE'})
lmap = lmap.rename(columns={'index': 'CELL_ID'})

# Select the appropriate columns.
concordance = lmap[['CELL_ID', 'SA2_ID']]

### Write concordance to CSV ###

In [4]:
concordance.to_csv('cells-sa2.csv', index=False)

## LANDUSES (lexicographically ordered list of land-uses)

In [5]:
# Distill the lexicographically ordered list of land-uses.
landuses = sorted(lmap['LU_DESC'].unique().to_list())
landuses.remove('Non-agricultural land') # Remove this non land-use.

# Distill a LU_ID to LU_DESC concordance table.
luid2desc = lmap.groupby('LU_ID').first()['LU_DESC']
luid2desc = luid2desc.astype(str)

### Write land-use list to CSV

In [6]:
pd.DataFrame(landuses).to_csv('landuses.csv', index=False, header=False)

### Write land-use list to NPY

In [7]:
np.save('landuses.npy', np.array(landuses))

### Write LU_ID to LU_DESC concordance to HDF5.

In [8]:
luid2desc.to_hdf('luid2desc.hdf5', key='luid2desc')

## LUMAP (land-use mapping) ##

In [9]:
# Find where the land-uses are on the map.
locations = lmap['LU_DESC'].to_numpy()

# Create a highpos-style lumap. Use -1 for anything _not_ in the land-use list.
lucodes = [-1 if (r not in landuses) else landuses.index(r) for r in locations]
lumap = np.array(lucodes, dtype=np.int8)

# Certain land-uses are not available.
# nal = landuses.index('Non-agricultural land')
# plf = landuses.index('Plantation forestry')
# lumap = np.where((lumap==nal) | (lumap==plf), -1, lumap)
lumap_pd = pd.DataFrame(lumap)

### Write lumap to CSV ###

In [10]:
lumap_pd = lumap_pd.rename(columns={0: 'j'})
lumap_pd.to_csv('lumap.csv', index=False)

### Write lumap to NPY ###

In [11]:
np.save('lumap.npy', lumap)

## LMMAP (land-management mapping) ##

In [12]:
# For now only 'rain-fed' ('dry' == 0) and 'irrigated' ('irr' == 1) available.
lmmap = lmap['IRRIGATION']
lmmap_np = lmmap.to_numpy()

### Write lmmap to CSV ###

In [13]:
lmmap.to_csv('lmmap.csv', index=False)

### Write lmmap to NPY ###

In [14]:
np.save('lmmap.npy', lmmap)

From `cell_zones_df.h5` the following are obtained:
* REAL_AREA -- hectares per given cell, corrected for projection.

In [15]:
zones = pd.read_hdf('cell_zones_df.h5')

## REAL_AREA (hectares per cell, projection corrected) ##

In [16]:
real_area = zones['CELL_HA']
real_area_np = real_area.to_numpy()

### Write real_area to CSV ###

In [17]:
real_area.to_csv('real-area.csv', index=False)

### Write real_area to NPY ###

In [18]:
np.save('real-area.npy', real_area_np)

## POTENTIAL_IRRIGATION_AREAS

In [19]:
potential_irrigation_areas = zones['POTENTIAL_IRRIGATION_AREAS'].to_numpy()

### Write potential_irrigation_areas to NPY

In [20]:
np.save('potential-irrigation-areas.npy', potential_irrigation_areas)

## TMATRIX

In [21]:
# Read in from-to costs in category-to-category format.
tmcat = pd.read_csv('tmatrix-categories.csv', index_col=0)
# Read the categories to land-uses concordance.
cat2lus = pd.read_csv('tmatrix-cat2lus.csv').to_numpy()
# Produce a dictionary for ease of look up.
l2c = dict([(row[1], row[0]) for row in cat2lus])
# Prepare indices.
indices = [(lu1, lu2) for lu1 in landuses for lu2 in landuses]
# Prepare the DataFrame.
t = pd.DataFrame(index=landuses, columns=landuses, dtype=np.float64)
# Fill the DataFrame.
for i in indices: t.loc[i] = tmcat.loc[l2c[i[0]], l2c[i[1]]]
# Switching to existing land use (i.e. not switching) does not cost anything.
for lu in landuses: t.loc[lu, lu] = 0
# Extract the actual tmatrix Numpy array.
tmatrix = t.to_numpy()

### Write TMATRIX to NPY

In [22]:
np.save('tmatrix.npy', tmatrix)

## DIV SPATIAL DATA

Obtained from:
* cell_livestock_data.h5
* SA2_crop_data.h5
* cell_biophysical_df.h5 (from cell_biophysical_df.pkl)

In [23]:
lvstk = pd.read_hdf('cell_livestock_data.h5')
crops = pd.read_hdf('SA2_crop_data.h5')
bioph = pd.read_hdf('cell_biophysical_df.h5')

In [24]:
feed_req = ( lvstk[['FEED_REQ', 'CELL_ID']]
             .groupby('CELL_ID')
             .first()['FEED_REQ']
             .to_numpy() )

In [25]:
pasture_kg_dm_ha = ( lvstk[['PASTURE_KG_DM_HA', 'CELL_ID']]
                     .groupby('CELL_ID')
                     .first()['PASTURE_KG_DM_HA']
                     .to_numpy() )

In [26]:
safe_pur_natl = ( lvstk[['SAFE_PUR_NATL', 'CELL_ID']]
                  .groupby('CELL_ID')
                  .first()['SAFE_PUR_NATL']
                  .to_numpy() )

In [27]:
safe_pur_modl = ( lvstk[['SAFE_PUR_MODL', 'CELL_ID']]
                  .groupby('CELL_ID')
                  .first()['SAFE_PUR_MODL']
                  .to_numpy() )

In [28]:
c = crops[['SA2_ID', 'WP']].groupby('SA2_ID').first()
wpc = concordance.merge( c
                       , left_on='SA2_ID'
                       , right_on='SA2_ID'
                       , how='left' )['WP']
wpl = lvstk['WP']
water_delivery_price = np.where(np.isnan(wpc), wpl, wpc)
water_licence_price = bioph['WATER_PRICE_ML_BOM'].to_numpy()

In [29]:
prec_over_175mm = bioph['AVG_GROW_SEAS_PREC_GE_175_MM_YR'].to_numpy()

### Write feed_req to NPY

In [30]:
np.save('feed-req', feed_req)

### Write pasture_kg_dm_ha to NPY

In [31]:
np.save('pasture-kg-dm-ha', pasture_kg_dm_ha)

### Write safe_pur_natl to NPY

In [32]:
np.save('safe-pur-natl', safe_pur_natl)

### Write safe_pur_modl to NPY

In [33]:
np.save('safe-pur-modl', safe_pur_modl)

### Write water_delivery_price to NPY

In [34]:
np.save('water-delivery-price.npy', water_delivery_price)

In [35]:
np.save('water-licence-price.npy', water_licence_price)

### Write prec_over_175mm to NPY

In [36]:
np.save('prec-over-175mm.npy', prec_over_175mm)

## EXCLUDE MATRICES

From `NLUM_SPREAD_LU_ID_Mapped_Concordance.h5` the following are obtained:

* x_mrj -- exclude matrices.

In [37]:
# Read the 'ultimate truth' table.
ut = pd.read_hdf('NLUM_SPREAD_LU_ID_Mapped_Concordance.h5')

# Turn it into a pivot table. Non-NaN entries are allowed land-uses in the SA2.
ut_ptable = ut.pivot_table( index='SA2_ID'
                          , columns=['IRRIGATION', 'LU_DESC']
                          )['LU_ID']
x_dry = concordance.merge( ut_ptable[0]
                         , left_on='SA2_ID', right_on='SA2_ID'
                         , how='left')
x_irr = concordance.merge( ut_ptable[1]
                         , left_on='SA2_ID', right_on='SA2_ID'
                         , how='left')
x_dry = x_dry.drop(columns=['CELL_ID', 'SA2_ID'])
x_irr = x_irr.drop(columns=['CELL_ID', 'SA2_ID'])

# Some land uses never occur at all
# like dry-land pears and rice, grazing on irrigated natural land.
for lu in landuses:
    if lu not in x_dry.columns:
        x_dry[lu] = np.nan
    if lu not in x_irr.columns:
        x_irr[lu] = np.nan

# 'Unallocated - modified land' can occur anywhere but is never irrigated.
x_dry['Unallocated - modified land'] = 22
x_irr['Unallocated - modified land'] = np.nan

# 'Unallocated - natural land' is never irrigated. Occurs where t_mrj allows it. 
x_dry['Unallocated - natural land'] = 23
x_irr['Unallocated - natural land'] = np.nan

# Ensure lexicographical order.
x_dry.sort_index(axis='columns', inplace=True)
x_irr.sort_index(axis='columns', inplace=True)

# Turn into Numpy arrays.
x_dry = x_dry.to_numpy()
x_irr = x_irr.to_numpy()

# Turn into 'boolean' arrays.
x_dry = np.where(np.isnan(x_dry), 0, 1)
x_irr = np.where(np.isnan(x_irr), 0, 1)

# The land uses in the 'cropping' categories.
cropping_lus = [ 'Hay'
               , 'Other non-cereal crops'
               , 'Summer cereals'
               , 'Summer legumes'
               , 'Summer oilseeds'
               , 'Winter cereals'
               , 'Winter legumes'
               , 'Winter oilseeds'
               , 'Cotton'
               , 'Rice'
               , 'Sugar'
               , 'Vegetables' ]
clus = [ landuses.index(lu) for lu in cropping_lus ]

# Dryland cropping only if precipitation is over 175mm in growth season.
x_dry[:, clus] *= prec_over_175mm[:, np.newaxis]

# Irrigated land-usage is only allowed in potential irrigation areas.
x_irr *= potential_irrigation_areas[:, np.newaxis]

# Stack for neoLUTO use.
x_mrj = np.stack((x_dry, x_irr)).astype(np.int8)

### Write exclude matrices to NPY

In [38]:
np.save('x-mrj.npy', x_mrj)

## WATER

In [39]:
# Select the required columns.
rivregs = zones[['HR_RIVREG_ID', 'HR_RIVREG_NAME']].copy()

# Make sure the cell ids are zero-based.
rivregs['LUTO_ID'] = rivregs.index

# Reorder columns.
rivregs = rivregs[['LUTO_ID', 'HR_RIVREG_ID', 'HR_RIVREG_NAME']]

# Select the required columns.
draindivs = zones[['HR_DRAINDIV_ID', 'HR_DRAINDIV_NAME']].copy()

# Make sure the cell ids are zero-based.
draindivs['LUTO_ID'] = draindivs.index

# Reorder columns.
draindivs = draindivs[['LUTO_ID', 'HR_DRAINDIV_ID', 'HR_DRAINDIV_NAME']]

In [40]:
water_yield_baselines = bioph[['WATER_YIELD_DR_ML_HA', 'WATER_YIELD_SR_ML_HA']]

### Write rivregs to HDF5

In [41]:
rivregs.to_hdf( 'rivregs.hdf5', key='rivregs'
              , mode='w', format='table', index=False )

### Write draindivs to HDF5

In [42]:
draindivs.to_hdf( 'draindivs.hdf5', key='draindivs'
                , mode='w', format='table', index=False )

### Write water_yield_baselines to HDF5 

In [43]:
water_yield_baselines.to_hdf( 'water-yield-baselines.hdf5', key='water_yieldbase_lines'
                            , mode='w', format='table', index=False )

## CLIMATE IMPACTS

In [ ]:
# Load raw climate impact data.
cci_raw = pd.read_hdf('SA2_climate_damage_mult.h5')

# Distill available RCPs.
rcps = {col[0] for col in cci_raw.columns}

for rcp in rcps:
    # Slice off RCP and turn into pivot table.
    cci_ptable = cci_raw[rcp].pivot_table( index='SA2_ID'
                                         , columns=['IRRIGATION', 'LU_ID'] )

    # Merge with SA2-Cell concordance to obtain cell-based pivot table.
    cci = concordance.merge( cci_ptable
                           , left_on='SA2_ID', right_on='SA2_ID'
                           , how='left')

    # Not all columns are needed.
    cci = cci.drop(['CELL_ID', 'SA2_ID'], axis=1)

    # Land-uses as strings and years as integers.
    lmid2desc = {0: 'dry', 1: 'irr'}
    coltups = [ (int(col[0][3:]), lmid2desc[col[1]], luid2desc[col[2]])
                for col in cci.columns ]
    mcolumns = pd.MultiIndex.from_tuples(coltups)
    cci.columns = mcolumns
    
    # Arrange levels of multi-index as (lm, lu, year).
    cci = cci.swaplevel(0,1, axis='columns')
    cci = cci.swaplevel(1,2, axis='columns')
    cci.sort_index(axis=1, inplace=True)
    
    # Write to HDF5 file.
    fname = 'climate-change-impacts-' + rcp + '.hdf5'
    kname = 'climate_change_impacts_' + rcp 
    cci.to_hdf( fname, key=kname , mode='w', format='table'
              , complevel=9, index=False )

/usr/lib64/python3.10/site-packages/pandas/core/frame.py:9191: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,3 on the right)
  return merge(


In [7]:
def build_agec_crops(concordance, data, columns=None, lus=None, lms=None):
    """Return LUTO-cell wise, multi-indexed column-landuse-landman DataFrame."""

    # If no list of columns is provided, infer it from provided data.
    if columns is None:
        columns = data.columns.to_list()

    # If no list of land uses provided, infer it from the SPREAD_CLASS column.
    if lus is None:
        lus = sorted(data['LU_DESC'].unique().tolist())

    # If no list of land managements provided, use irrigation status only.
    if lms is None:
        lms = ['dry', 'irr']

    # Produce a multi-indexed version data and merge to yield cell-based table.
    crops_ptable = data.pivot_table( index='SA2_ID'
                                   , columns=['Irrigation', 'LU_DESC'] )
    agec_crops = concordance.merge( crops_ptable
                                  , left_on='SA2_ID'
                                  , right_on=crops_ptable.index
                                  , how='left' )

    # Some columns have to go.
    agec_crops = agec_crops.drop(['CELL_ID', 'SA2_ID'], axis=1)

    # The merge flattens the multi-index to tuples, so unflatten here.
    ts = [(t[0], lms[t[1]], t[2]) for t in agec_crops.columns]
    agec_crops.columns = pd.MultiIndex.from_tuples(ts)

    return agec_crops

In [6]:
def build_agec_lvstk(data):
    """Return LUTO-cell wise, multi-indexed column-landuse DataFrame."""

    # Get only the columns that vary with the land-use.
    animals = [ c for c in data.columns if 'BEEF' in c
                                        or 'SHEEP' in c
                                        or 'DAIRY' in c ]
    agec_lvstk = data[animals]

    # Prepare columns for multi-indexing, i.e. turn into a list of tuples.
    cols = agec_lvstk.columns.to_list()
    cols = [tuple(c.split(sep='_')) for c in cols]
    cols = [(c[0]+'_'+c[1], c[2]) if c[0]=='WR' else c for c in cols]
    cols = [ (c[0] + '_' + c[1] + '_' + c[3] + '_' +c[4], c[2])
             if c[0]=='FEED' else c for c in cols ]

    # Make and set the multi-index.
    mcolindex = pd.MultiIndex.from_tuples(cols)
    agec_lvstk.columns = mcolindex

    return agec_lvstk

In [11]:
agec_crops = build_agec_crops(concordance, crops)
agec_lvstk = build_agec_lvstk(lvstk)

NameError: name 'concordance' is not defined

In [ ]:
agec_crops.to_hdf( 'input/agec-crops-c9.hdf5', 'agec_crops'
                 , mode='w'
                 , format='table'
                 , complevel=9
                 , index=False )

In [ ]:
agec_lvstk.to_hdf( 'agec-lvstk-c9.hdf5', 'agec_lvstk'
                 , mode='w'
                 , format='table'
                 , complevel=9
                 , index=False )